In [1]:
import prettytable
import tensorflow as tf
from keras import callbacks, initializers, optimizers, regularizers
from keras.layers import (LSTM, Bidirectional, Concatenate, Conv1D, Dense,
                          Dropout, Flatten, Input, MaxPooling1D,
                          TimeDistributed)
from keras.models import Model, Sequential
from sklearn.metrics import (accuracy_score, f1_score, hamming_loss,
                             precision_recall_curve, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from keras.models import load_model


In [2]:
tf.__version__

'2.3.0'

In [3]:
import pandas as pd
import numpy as np
import datetime

In [4]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [5]:
# Loading Tensorboard Extension
%load_ext tensorboard

In [6]:
from MultiChannelBiLSTMCNN import BalanceNet, time_logger_save


In [7]:
# Metrics Calculator Function
def evaluate_model(real, predicted):
    accuracy = accuracy_score(real, predicted)
    hamLoss = hamming_loss(real, predicted)
    # element wise correctness
    term_wise_accuracy = np.sum(np.logical_not(
        np.logical_xor(real, predicted)))/real.size

    macro_precision = precision_score(real, predicted, average='macro')
    macro_recall = recall_score(real, predicted, average='macro')
    macro_f1 = f1_score(real, predicted, average='macro')

    micro_precision = precision_score(real, predicted, average='micro')
    micro_recall = recall_score(real, predicted, average='micro')
    micro_f1 = f1_score(real, predicted, average='micro')

    metricTable = prettytable.PrettyTable()
    metricTable.field_names = ["Metric", "Macro Value", "Micro Value"]
    metricTable.add_row(["Hamming Loss", "{0:.3f}".format(hamLoss), ""])
    metricTable.add_row(
        ["Term Wise Accuracy", "{0:.3f}".format(term_wise_accuracy), ""])

    metricTable.add_row(["Accuracy", "{0:.3f}".format(accuracy), ""])
    metricTable.add_row(["Precision", "{0:.3f}".format(
        macro_precision), "{0:.3f}".format(micro_precision)])
    metricTable.add_row(["Recall", "{0:.3f}".format(
        macro_recall), "{0:.3f}".format(micro_recall)])
    metricTable.add_row(
        ["F1-measure", "{0:.3f}".format(macro_f1), "{0:.3f}".format(micro_f1)])

    print(metricTable)

In [8]:
#Helper Functions
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
  
def initial_boost(epoch):
    if epoch==0: return float(8.0)
    elif epoch==1: return float(4.0)
    elif epoch==2: return float(2.0)
    elif epoch==3: return float(1.5)
    else: return float(1.0)

def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.0001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        #print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)


Loading Data

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
mpstDF= pd.read_csv("/content/drive/My Drive/Colab Notebooks/Embeddings/mpst.csv")
mpstDF

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb
...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia


In [11]:
# Data Split Function
def get_partition_Embeddings(x_t1,x_t2,y,df,partition_nm):
    _df=df[df["split"]==partition_nm]
    index_list=list(_df.index)
    temp_array_x_t1=[]
    temp_array_x_t2=[]
    temp_array_y=[]
    for index in index_list:
        temp_array_x_t1.append(x_t1[index,:])
        temp_array_x_t2.append(x_t2[index,:])
        temp_array_y.append(y[index,:])
    temp_array_x_t1=np.array(temp_array_x_t1)
    temp_array_x_t2=np.array(temp_array_x_t2)
    temp_array_y=np.array(temp_array_y)
    return temp_array_x_t1,temp_array_x_t2, temp_array_y

In [12]:
# LOADING BERT  EMBEDDINGS
bert_embedding=np.load("/content/drive/My Drive/Colab Notebooks/Embeddings/embeddings.npz")
# LOADING XLnet EMBEDDINGS
xl_embedding=np.load("/content/drive/My Drive/Colab Notebooks/Embeddings/xl_embeddings.npz")
# LOADING LABELS Y
label_values=np.load("/content/drive/My Drive/Colab Notebooks/Embeddings/Y.npz")

In [13]:
# BERT EMBEDDINGS T1 & T2
type1_BERT_Embeddings=bert_embedding["t1"]
type2_BERT_Embeddings=bert_embedding["t2"]
# XLNet EMBEDDINGS T1 & T2
type1_XL_Embeddings=xl_embedding["t1"]
type2_XL_Embeddings=xl_embedding["t2"]
# LABLES Y
label_values=label_values["arr_0"]

In [14]:
# BERT

# FOR TRAIN
type1_BERT_Embeddings_Train,type2_BERT_Embeddings_Train,BERT_label_values_Train=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"train")
# FOR VALIDATION
type1_BERT_Embeddings_Val,type2_BERT_Embeddings_Val,BERT_label_values_Val=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"val")
# FOR TEST
type1_BERT_Embeddings_Test,type2_BERT_Embeddings_Test,BERT_label_values_Test=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"test")

In [15]:
# XLNET

# FOR TRAIN
type1_XL_Embeddings_Train,type2_XL_Embeddings_Train,XLNET_label_values_Train=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"train")
# FOR VALIDATION
type1_XL_Embeddings_Val,type2_XL_Embeddings_Val,XLNET_label_values_Val=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"val")
# FOR TEST
type1_XL_Embeddings_Test,type2_XL_Embeddings_Test,XLNET_label_values_Test=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"test")

HAVING A LOOK AT THE SHAPES OF EACH PARTITION CREATED:

In [16]:
print("SHAPES OF EACH PARTITION _ BERT\n")
print("type1_BERT_Embeddings_Train.shape: ", type1_BERT_Embeddings_Train.shape)
print("type2_BERT_Embeddings_Train.shape: ", type2_BERT_Embeddings_Train.shape)
print("BERT_label_values_Train.shape: "    , BERT_label_values_Train.shape)

print("type1_BERT_Embeddings_Val.shape: ", type1_BERT_Embeddings_Val.shape)
print("type2_BERT_Embeddings_Val.shape: ", type2_BERT_Embeddings_Val.shape)
print("BERT_label_values_Val.shape: "    , BERT_label_values_Val.shape)

print("type1_BERT_Embeddings_Test.shape: ", type1_BERT_Embeddings_Test.shape)
print("type2_BERT_Embeddings_Test.shape: ", type2_BERT_Embeddings_Test.shape)
print("BERT_label_values_Test.shape: "    , BERT_label_values_Test.shape)

print("SHAPES OF EACH PARTITION _ XLNET\n")

print("type1_XL_Embeddings_Train.shape: ", type1_XL_Embeddings_Train.shape)
print("type2_XL_Embeddings_Train.shape: ", type2_XL_Embeddings_Train.shape)
print("XLNET_label_values_Train.shape: " , XLNET_label_values_Train.shape)

print("type1_XL_Embeddings_Val.shape: ", type1_XL_Embeddings_Val.shape)
print("type2_XL_Embeddings_Val.shape: ", type2_XL_Embeddings_Val.shape)
print("XLNET_label_values_Val.shape: " , XLNET_label_values_Val.shape)

print("type1_XL_Embeddings_Test.shape: ", type1_XL_Embeddings_Test.shape)
print("type2_XL_Embeddings_Test.shape: ", type2_XL_Embeddings_Test.shape)
print("XLNET_label_values_Test.shape: " , XLNET_label_values_Test.shape)

print("SHAPES OF EACH PARTITION _ BERT\n")
print("type1_BERT_Embeddings_Train.: ", type1_BERT_Embeddings_Train)
print("type2_BERT_Embeddings_Train.: ", type2_BERT_Embeddings_Train)
print("BERT_label_values_Train.: "    , BERT_label_values_Train)

print("type1_BERT_Embeddings_Val.: ", type1_BERT_Embeddings_Val)
print("type2_BERT_Embeddings_Val.: ", type2_BERT_Embeddings_Val)
print("BERT_label_values_Val.: "    , BERT_label_values_Val)

print("type1_BERT_Embeddings_Test.shape: ", type1_BERT_Embeddings_Test)
print("type2_BERT_Embeddings_Test.shape: ", type2_BERT_Embeddings_Test)
print("BERT_label_values_Test.shape: "    , BERT_label_values_Test)

print("SHAPES OF EACH PARTITION _ XLNET\n")

print("type1_XL_Embeddings_Train: ", type1_XL_Embeddings_Train)
print("type2_XL_Embeddings_Train: ", type2_XL_Embeddings_Train)
print("XLNET_label_values_Train: " , XLNET_label_values_Train)

print("type1_XL_Embeddings_Val: ", type1_XL_Embeddings_Val)
print("type2_XL_Embeddings_Val: ", type2_XL_Embeddings_Val)
print("XLNET_label_values_Val: " , XLNET_label_values_Val)

print("type1_XL_Embeddings_Test: ", type1_XL_Embeddings_Test)
print("type2_XL_Embeddings_Test: ", type2_XL_Embeddings_Test)
print("XLNET_label_values_Test: " , XLNET_label_values_Test)


SHAPES OF EACH PARTITION _ BERT

type1_BERT_Embeddings_Train.shape:  (9489, 768)
type2_BERT_Embeddings_Train.shape:  (9489, 768)
BERT_label_values_Train.shape:  (9489, 71)
type1_BERT_Embeddings_Val.shape:  (2373, 768)
type2_BERT_Embeddings_Val.shape:  (2373, 768)
BERT_label_values_Val.shape:  (2373, 71)
type1_BERT_Embeddings_Test.shape:  (2966, 768)
type2_BERT_Embeddings_Test.shape:  (2966, 768)
BERT_label_values_Test.shape:  (2966, 71)
SHAPES OF EACH PARTITION _ XLNET

type1_XL_Embeddings_Train.shape:  (9489, 768)
type2_XL_Embeddings_Train.shape:  (9489, 768)
XLNET_label_values_Train.shape:  (9489, 71)
type1_XL_Embeddings_Val.shape:  (2373, 768)
type2_XL_Embeddings_Val.shape:  (2373, 768)
XLNET_label_values_Val.shape:  (2373, 71)
type1_XL_Embeddings_Test.shape:  (2966, 768)
type2_XL_Embeddings_Test.shape:  (2966, 768)
XLNET_label_values_Test.shape:  (2966, 71)
SHAPES OF EACH PARTITION _ BERT

type1_BERT_Embeddings_Train.:  [[-0.23652047 -0.46730292  0.62800884 ... -0.3478354   0.17314

In [17]:
# function to reshape my input
def tensor_reshape(input, timestep, features):
  reshaped= input.reshape(type1_BERT_Embeddings_Train.shape[0], timestep, features)
  return reshaped

STARTING THE MODEL
possible combination in inputs:
(timestep, features)
- (1, 768)
- (2, 384)
- (3, 256)
- (4, 192)
- (6, 128)
- (12, 64)

## MODEL (1, 768):


In [18]:
INPUT_SHAPE =(1,768)
EM_L_F_UNITS= 768
EM_L_T_UNITS= 768
# LEFT CHANNEL
LSTM_1F_UNITS= 128
LSTM_1T_UNITS= 128

CONV_2_FILTER= 24
CONV_2_KERNEL= 2
CONV_3_FILTER= 24
CONV_3_KERNEL= 2
CONV_5_FILTER= 24
CONV_5_KERNEL= 2
CONV_6_FILTER= 24
CONV_6_KERNEL= 2
CONV_8_FILTER= 24
CONV_8_KERNEL= 2

# RIGHT CHANNEL 
CONV_4F_FILTERS = 12
CONV_4F_KERNEL = 1
CONV_4T_FILTERS = 12
CONV_4T_KERNEL = 1

CONV_3F_FILTERS = 12
CONV_3F_KERNEL = 1
CONV_3T_FILTERS = 12
CONV_3T_KERNEL = 1

CONV_2F_FILTERS = 12
CONV_2F_KERNEL = 1
CONV_2T_FILTERS = 12
CONV_2T_KERNEL = 1

LSTM_2_C_L_UNITS = 12

OUTPUT_DENSE_UNIT =128
OUTPUT_SIZE =71



In [19]:
optimizer_list = ['adam']
dataset_X={
    "bert_t1":[
        type1_BERT_Embeddings_Train,
        type1_BERT_Embeddings_Val,
        type1_BERT_Embeddings_Test
        ],
    "bert_t2":[
        type2_BERT_Embeddings_Train,
        type2_BERT_Embeddings_Val,
        type2_BERT_Embeddings_Test
        ],
    "xlnet_t1":[
        type1_XL_Embeddings_Train,
        type1_XL_Embeddings_Val,
        type1_XL_Embeddings_Test
        ],
    "xlnet_t2":[
        type2_XL_Embeddings_Train,
        type2_XL_Embeddings_Val,
        type2_XL_Embeddings_Test
    ]
}
dataset_Y ={
    "bert":[
        BERT_label_values_Train,
        BERT_label_values_Val,
        BERT_label_values_Test
        ],
    "xlnet":[
        XLNET_label_values_Train,
        XLNET_label_values_Val,
        XLNET_label_values_Test
        ]
}

inp_shape_str = "1_768"


In [20]:
model_dict ={}
t_list =[x/10 for x in range(1,10)]
time_dict=[]

for ds in dataset_X.keys():
    for opt in optimizer_list:
        print("\nFOR OPTIMIZER: ",opt)
        key= "model_"+inp_shape_str+"_"+opt+"_"+ds
        model_type = ds+"_"+opt
        print("\n###############\nKEY= "+key+"\n#############")
        print("\n###############\nmodel_type= "+model_type+"\n#############")
        model_dict[key] = BalanceNet(INPUT_SHAPE,
                                    EM_L_F_UNITS,
                                    EM_L_T_UNITS,
                                    LSTM_1F_UNITS,
                                    LSTM_1T_UNITS,
                                    CONV_2_FILTER,
                                    CONV_2_KERNEL,
                                    CONV_3_FILTER,
                                    CONV_3_KERNEL,
                                    CONV_5_FILTER,
                                    CONV_5_KERNEL,
                                    CONV_6_FILTER,
                                    CONV_6_KERNEL,
                                    CONV_8_FILTER,
                                    CONV_8_KERNEL,
                                    CONV_4F_FILTERS,
                                    CONV_4F_KERNEL,
                                    CONV_4T_FILTERS,
                                    CONV_4T_KERNEL,
                                    CONV_3F_FILTERS,
                                    CONV_3F_KERNEL,
                                    CONV_3T_FILTERS,
                                    CONV_3T_KERNEL,
                                    CONV_2F_FILTERS,
                                    CONV_2F_KERNEL,
                                    CONV_2T_FILTERS,
                                    CONV_2T_KERNEL,
                                    LSTM_2_C_L_UNITS,
                                    OUTPUT_DENSE_UNIT,
                                    OUTPUT_SIZE,
                                    optimizer_name= opt)

        if 'bert' in ds:
            em_type='bert'
        if 'xlnet' in ds:
            em_type='xlnet'
        model_dict[key].fitModel(train_x =dataset_X[ds][0], 
                                            train_y=dataset_Y[em_type][0], 
                                            val_x= dataset_X[ds][1], 
                                            val_y = dataset_Y[em_type][1], 
                                            model_type = model_type, 
                                            epochs_count = 20 , 
                                            batch_count = 64,  time_dict=time_dict, key=key)
        model_dict[key].predictModel(test_x=dataset_X[ds][2], test_y=dataset_Y[em_type][2], threshold_list=t_list,  time_dict=time_dict, key=key)
time_logger_save(time_dict,inp_shape_str+"_time")


FOR OPTIMIZER:  adam

###############
KEY= model_1_768_adam_bert_t1
#############

###############
model_type= bert_t1_adam
#############
inp_layer (None, 1, 768)
embedding_layer_frozen (None, 1, 768)
embedding_layer_train (None, 1, 768)
l_lstm_1f (None, 1, 256)
l_lstm_1t (None, 1, 256)
l_lstm1 (None, 2, 256)
l_conv_2 (None, 1, 24)
l_conv_3 (None, 1, 24)
l_conv_5 (None, 1, 24)
l_conv_6 (None, 1, 24)
l_conv_8 (None, 1, 24)
l_lstm_c (None, 5, 24)
l_conv_4f (None, 1, 12)
l_conv_4t (None, 1, 12)
l_conv_3f (None, 1, 12)
l_conv_3t (None, 1, 12)
l_conv_2f (None, 1, 12)
l_conv_2t (None, 1, 12)
l_merge_2 (None, 6, 12)
l_c_lstm (None, 6, 24)
Training Progress for: bert_t1_adam
Reshaping inputs
Reshaped Into Shape:  (9489, 1, 768)
Reshaped Into Shape:  (2373, 1, 768)
Reshaped Successfully!
Epoch 1/20
149/149 [==============================] - ETA: 0s - loss: 10.5740 - acc: 0.1201 - categorical_accuracy: 0.1201 - top_k_categorical_accuracy: 0.3830 - precision: 0.1999 - recall: 0.5406 - true_posit

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


count_1_as_1, TP 6669
count_1_as_0, FN 2353
count_0_as_1, FP 46719
count_0_as_0, TN 154845
total_real_1s 9022
total_real_0s 201564
+--------------------+-------------+-------------+
|       Metric       | Macro Value | Micro Value |
+--------------------+-------------+-------------+
|    Hamming Loss    |    0.233    |             |
| Term Wise Accuracy |    0.767    |             |
|      Accuracy      |    0.000    |             |
|     Precision      |    0.032    |    0.125    |
|       Recall       |    0.254    |    0.739    |
|     F1-measure     |    0.053    |    0.214    |
+--------------------+-------------+-------------+
+-------------------------------+--------+
|             Metric            | Value  |
+-------------------------------+--------+
|  True Positives, count_1_as_1 |  6669  |
| False Negatives, count_1_as_0 |  2353  |
| False Positives, count_0_as_1 | 46719  |
|  True Negatives, count_0_as_0 | 154845 |
|            Real 1s            |  9022  |
|            Re

### END

## MODEL (2,384)

In [21]:
INPUT_SHAPE =(2,384)
EM_L_F_UNITS= 384
EM_L_T_UNITS= 384
# LEFT CHANNEL
LSTM_1F_UNITS= 128
LSTM_1T_UNITS= 128

CONV_2_FILTER= 64
CONV_2_KERNEL= 1
CONV_3_FILTER= 64
CONV_3_KERNEL= 2
CONV_5_FILTER= 64
CONV_5_KERNEL= 3
CONV_6_FILTER= 64
CONV_6_KERNEL= 2
CONV_8_FILTER= 64
CONV_8_KERNEL= 3

# RIGHT CHANNEL 
CONV_4F_FILTERS = 64
CONV_4F_KERNEL = 2
CONV_4T_FILTERS = 64
CONV_4T_KERNEL = 2

CONV_3F_FILTERS = 64
CONV_3F_KERNEL = 2
CONV_3T_FILTERS = 64
CONV_3T_KERNEL = 2

CONV_2F_FILTERS = 64
CONV_2F_KERNEL = 2
CONV_2T_FILTERS = 64
CONV_2T_KERNEL = 2

LSTM_2_C_L_UNITS = 32

OUTPUT_DENSE_UNIT =128
OUTPUT_SIZE =71

inp_shape_str = "2_384"



## RUNNING MODELS FOR 2_384

In [22]:
model_dict ={}
t_list =[x/10 for x in range(1,10)]
time_dict=[]

for ds in dataset_X.keys():
    print("\nFOR DATASET: ", ds)
    for opt in optimizer_list:
        print("FOR OPTIMIZER: ",opt)
        key= "model_"+inp_shape_str+"_"+opt+"_"+ds
        model_type = ds+"_"+opt
        print("\n###############\nKEY= "+key+"\n#############")
        print("\n###############\nmodel_type= "+model_type+"\n#############")
        model_dict[key] = BalanceNet(INPUT_SHAPE,
                                    EM_L_F_UNITS,
                                    EM_L_T_UNITS,
                                    LSTM_1F_UNITS,
                                    LSTM_1T_UNITS,
                                    CONV_2_FILTER,
                                    CONV_2_KERNEL,
                                    CONV_3_FILTER,
                                    CONV_3_KERNEL,
                                    CONV_5_FILTER,
                                    CONV_5_KERNEL,
                                    CONV_6_FILTER,
                                    CONV_6_KERNEL,
                                    CONV_8_FILTER,
                                    CONV_8_KERNEL,
                                    CONV_4F_FILTERS,
                                    CONV_4F_KERNEL,
                                    CONV_4T_FILTERS,
                                    CONV_4T_KERNEL,
                                    CONV_3F_FILTERS,
                                    CONV_3F_KERNEL,
                                    CONV_3T_FILTERS,
                                    CONV_3T_KERNEL,
                                    CONV_2F_FILTERS,
                                    CONV_2F_KERNEL,
                                    CONV_2T_FILTERS,
                                    CONV_2T_KERNEL,
                                    LSTM_2_C_L_UNITS,
                                    OUTPUT_DENSE_UNIT,
                                    OUTPUT_SIZE,
                                    optimizer_name= opt)
        if 'bert' in ds:
            em_type='bert'
        if 'xlnet' in ds:
            em_type='xlnet'
        model_dict[key].fitModel(train_x =dataset_X[ds][0], 
                                            train_y=dataset_Y[em_type][0], 
                                            val_x= dataset_X[ds][1], 
                                            val_y = dataset_Y[em_type][1], 
                                            model_type = model_type, 
                                            epochs_count = 20 , 
                                            batch_count = 64,  time_dict=time_dict, key=key)
        model_dict[key].predictModel(test_x=dataset_X[ds][2], test_y=dataset_Y[em_type][2], threshold_list=t_list,  time_dict=time_dict, key=key)
time_logger_save(time_dict,inp_shape_str+"_time")



FOR DATASET:  bert_t1
FOR OPTIMIZER:  adam

###############
KEY= model_2_384_adam_bert_t1
#############

###############
model_type= bert_t1_adam
#############
inp_layer (None, 2, 384)
embedding_layer_frozen (None, 2, 384)
embedding_layer_train (None, 2, 384)
l_lstm_1f (None, 2, 256)
l_lstm_1t (None, 2, 256)
l_lstm1 (None, 4, 256)
l_conv_2 (None, 4, 64)
l_conv_3 (None, 3, 64)
l_conv_5 (None, 2, 64)
l_conv_6 (None, 3, 64)
l_conv_8 (None, 2, 64)
l_lstm_c (None, 14, 64)
l_conv_4f (None, 1, 64)
l_conv_4t (None, 1, 64)
l_conv_3f (None, 1, 64)
l_conv_3t (None, 1, 64)
l_conv_2f (None, 1, 64)
l_conv_2t (None, 1, 64)
l_merge_2 (None, 6, 64)
l_c_lstm (None, 6, 64)
Training Progress for: bert_t1_adam
Reshaping inputs
Reshaped Into Shape:  (9489, 2, 384)
Reshaped Into Shape:  (2373, 2, 384)
Reshaped Successfully!
Epoch 1/20
149/149 [==============================] - ETA: 0s - loss: 10.6321 - acc: 0.1099 - categorical_accuracy: 0.1099 - top_k_categorical_accuracy: 0.4011 - precision_4: 0.1937 - re

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


count_1_as_1, TP 7211
count_1_as_0, FN 1811
count_0_as_1, FP 63973
count_0_as_0, TN 137591
total_real_1s 9022
total_real_0s 201564
+--------------------+-------------+-------------+
|       Metric       | Macro Value | Micro Value |
+--------------------+-------------+-------------+
|    Hamming Loss    |    0.312    |             |
| Term Wise Accuracy |    0.688    |             |
|      Accuracy      |    0.000    |             |
|     Precision      |    0.034    |    0.101    |
|       Recall       |    0.338    |    0.799    |
|     F1-measure     |    0.058    |    0.180    |
+--------------------+-------------+-------------+
+-------------------------------+--------+
|             Metric            | Value  |
+-------------------------------+--------+
|  True Positives, count_1_as_1 |  7211  |
| False Negatives, count_1_as_0 |  1811  |
| False Positives, count_0_as_1 | 63973  |
|  True Negatives, count_0_as_0 | 137591 |
|            Real 1s            |  9022  |
|            Re

### END

# MODEL (3 ,256)

In [23]:
INPUT_SHAPE =(3, 256)
EM_L_F_UNITS= 256
EM_L_T_UNITS= 256
# LEFT CHANNEL
LSTM_1F_UNITS= 256
LSTM_1T_UNITS= 256

CONV_2_FILTER= 64
CONV_2_KERNEL= 2
CONV_3_FILTER= 64
CONV_3_KERNEL= 3
CONV_5_FILTER= 64
CONV_5_KERNEL= 4
CONV_6_FILTER= 64
CONV_6_KERNEL= 5
CONV_8_FILTER= 64
CONV_8_KERNEL= 6

# RIGHT CHANNEL 
CONV_4F_FILTERS = 64
CONV_4F_KERNEL = 3
CONV_4T_FILTERS = 64
CONV_4T_KERNEL = 3

CONV_3F_FILTERS = 64
CONV_3F_KERNEL = 3
CONV_3T_FILTERS = 64
CONV_3T_KERNEL = 3

CONV_2F_FILTERS = 64
CONV_2F_KERNEL = 3
CONV_2T_FILTERS = 64
CONV_2T_KERNEL = 3

LSTM_2_C_L_UNITS = 32

OUTPUT_DENSE_UNIT =128
OUTPUT_SIZE =71
inp_shape_str = "3_256"


## RUNNING MODELS FOR 3_256

In [24]:
model_dict ={}
t_list =[x/10 for x in range(1,10)]
time_dict=[]

for ds in dataset_X.keys():
    print("\nFOR DATASET: ", ds)
    for opt in optimizer_list:
        print("FOR OPTIMIZER: ",opt)
        key= "model_"+inp_shape_str+"_"+opt+"_"+ds
        model_type = ds+"_"+opt
        print("\n###############\nKEY= "+key+"\n#############")
        print("\n###############\nmodel_type= "+model_type+"\n#############")
        model_dict[key] = BalanceNet(INPUT_SHAPE,
                                    EM_L_F_UNITS,
                                    EM_L_T_UNITS,
                                    LSTM_1F_UNITS,
                                    LSTM_1T_UNITS,
                                    CONV_2_FILTER,
                                    CONV_2_KERNEL,
                                    CONV_3_FILTER,
                                    CONV_3_KERNEL,
                                    CONV_5_FILTER,
                                    CONV_5_KERNEL,
                                    CONV_6_FILTER,
                                    CONV_6_KERNEL,
                                    CONV_8_FILTER,
                                    CONV_8_KERNEL,
                                    CONV_4F_FILTERS,
                                    CONV_4F_KERNEL,
                                    CONV_4T_FILTERS,
                                    CONV_4T_KERNEL,
                                    CONV_3F_FILTERS,
                                    CONV_3F_KERNEL,
                                    CONV_3T_FILTERS,
                                    CONV_3T_KERNEL,
                                    CONV_2F_FILTERS,
                                    CONV_2F_KERNEL,
                                    CONV_2T_FILTERS,
                                    CONV_2T_KERNEL,
                                    LSTM_2_C_L_UNITS,
                                    OUTPUT_DENSE_UNIT,
                                    OUTPUT_SIZE,
                                    optimizer_name= opt)
        if 'bert' in ds:
            em_type='bert'
        if 'xlnet' in ds:
            em_type='xlnet'
        model_dict[key].fitModel(train_x =dataset_X[ds][0], 
                                            train_y=dataset_Y[em_type][0], 
                                            val_x= dataset_X[ds][1], 
                                            val_y = dataset_Y[em_type][1], 
                                            model_type = model_type, 
                                            epochs_count = 20 , 
                                            batch_count = 64,  time_dict=time_dict, key=key)
        model_dict[key].predictModel(test_x=dataset_X[ds][2], test_y=dataset_Y[em_type][2], threshold_list=t_list,  time_dict=time_dict, key=key)
time_logger_save(time_dict,inp_shape_str+"_time")



FOR DATASET:  bert_t1
FOR OPTIMIZER:  adam

###############
KEY= model_3_256_adam_bert_t1
#############

###############
model_type= bert_t1_adam
#############
inp_layer (None, 3, 256)
embedding_layer_frozen (None, 3, 256)
embedding_layer_train (None, 3, 256)
l_lstm_1f (None, 3, 512)
l_lstm_1t (None, 3, 512)
l_lstm1 (None, 6, 512)
l_conv_2 (None, 5, 64)
l_conv_3 (None, 4, 64)
l_conv_5 (None, 3, 64)
l_conv_6 (None, 2, 64)
l_conv_8 (None, 1, 64)
l_lstm_c (None, 15, 64)
l_conv_4f (None, 1, 64)
l_conv_4t (None, 1, 64)
l_conv_3f (None, 1, 64)
l_conv_3t (None, 1, 64)
l_conv_2f (None, 1, 64)
l_conv_2t (None, 1, 64)
l_merge_2 (None, 6, 64)
l_c_lstm (None, 6, 64)
Training Progress for: bert_t1_adam
Reshaping inputs
Reshaped Into Shape:  (9489, 3, 256)
Reshaped Into Shape:  (2373, 3, 256)
Reshaped Successfully!
Epoch 1/20
149/149 [==============================] - ETA: 0s - loss: 10.6434 - acc: 0.1405 - categorical_accuracy: 0.1405 - top_k_categorical_accuracy: 0.4193 - precision_8: 0.1941 - re

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


count_1_as_1, TP 7672
count_1_as_0, FN 1350
count_0_as_1, FP 75376
count_0_as_0, TN 126188
total_real_1s 9022
total_real_0s 201564
+--------------------+-------------+-------------+
|       Metric       | Macro Value | Micro Value |
+--------------------+-------------+-------------+
|    Hamming Loss    |    0.364    |             |
| Term Wise Accuracy |    0.636    |             |
|      Accuracy      |    0.000    |             |
|     Precision      |    0.036    |    0.092    |
|       Recall       |    0.394    |    0.850    |
|     F1-measure     |    0.062    |    0.167    |
+--------------------+-------------+-------------+
+-------------------------------+--------+
|             Metric            | Value  |
+-------------------------------+--------+
|  True Positives, count_1_as_1 |  7672  |
| False Negatives, count_1_as_0 |  1350  |
| False Positives, count_0_as_1 | 75376  |
|  True Negatives, count_0_as_0 | 126188 |
|            Real 1s            |  9022  |
|            Re

### END

## MODEL (4, 192)

In [25]:
INPUT_SHAPE =(4, 192)
EM_L_F_UNITS= 192
EM_L_T_UNITS= 192
# LEFT CHANNEL
LSTM_1F_UNITS= 256
LSTM_1T_UNITS= 256

CONV_2_FILTER= 128
CONV_2_KERNEL= 2
CONV_3_FILTER= 128
CONV_3_KERNEL= 3
CONV_5_FILTER= 128
CONV_5_KERNEL= 4
CONV_6_FILTER= 128
CONV_6_KERNEL= 6
CONV_8_FILTER= 128
CONV_8_KERNEL= 8

# RIGHT CHANNEL 
CONV_4F_FILTERS = 128
CONV_4F_KERNEL = 4
CONV_4T_FILTERS = 128
CONV_4T_KERNEL = 4

CONV_3F_FILTERS = 128
CONV_3F_KERNEL = 3
CONV_3T_FILTERS = 128
CONV_3T_KERNEL = 3

CONV_2F_FILTERS = 128
CONV_2F_KERNEL = 2
CONV_2T_FILTERS = 128
CONV_2T_KERNEL = 2

LSTM_2_C_L_UNITS = 64

OUTPUT_DENSE_UNIT =128
OUTPUT_SIZE =71
inp_shape_str = "4_192"


## RUNNING MODELS FOR 4_192

In [26]:
model_dict ={}
t_list =[x/10 for x in range(1,10)]
time_dict=[]

for ds in dataset_X.keys():
    print("\nFOR DATASET: ", ds)
    for opt in optimizer_list:
        print("FOR OPTIMIZER: ",opt)
        key= "model_"+inp_shape_str+"_"+opt+"_"+ds
        model_type = ds+"_"+opt
        print("\n###############\nKEY= "+key+"\n#############")
        print("\n###############\nmodel_type= "+model_type+"\n#############")
        model_dict[key] = BalanceNet(INPUT_SHAPE,
                                    EM_L_F_UNITS,
                                    EM_L_T_UNITS,
                                    LSTM_1F_UNITS,
                                    LSTM_1T_UNITS,
                                    CONV_2_FILTER,
                                    CONV_2_KERNEL,
                                    CONV_3_FILTER,
                                    CONV_3_KERNEL,
                                    CONV_5_FILTER,
                                    CONV_5_KERNEL,
                                    CONV_6_FILTER,
                                    CONV_6_KERNEL,
                                    CONV_8_FILTER,
                                    CONV_8_KERNEL,
                                    CONV_4F_FILTERS,
                                    CONV_4F_KERNEL,
                                    CONV_4T_FILTERS,
                                    CONV_4T_KERNEL,
                                    CONV_3F_FILTERS,
                                    CONV_3F_KERNEL,
                                    CONV_3T_FILTERS,
                                    CONV_3T_KERNEL,
                                    CONV_2F_FILTERS,
                                    CONV_2F_KERNEL,
                                    CONV_2T_FILTERS,
                                    CONV_2T_KERNEL,
                                    LSTM_2_C_L_UNITS,
                                    OUTPUT_DENSE_UNIT,
                                    OUTPUT_SIZE,
                                    optimizer_name= opt)
        if 'bert' in ds:
            em_type='bert'
        if 'xlnet' in ds:
            em_type='xlnet'
        model_dict[key].fitModel(train_x =dataset_X[ds][0], 
                                            train_y=dataset_Y[em_type][0], 
                                            val_x= dataset_X[ds][1], 
                                            val_y = dataset_Y[em_type][1], 
                                            model_type = model_type, 
                                            epochs_count = 20 , 
                                            batch_count = 64,  time_dict=time_dict, key=key)
        model_dict[key].predictModel(test_x=dataset_X[ds][2], test_y=dataset_Y[em_type][2], threshold_list=t_list,  time_dict=time_dict, key=key)
time_logger_save(time_dict,inp_shape_str+"_time")



FOR DATASET:  bert_t1
FOR OPTIMIZER:  adam

###############
KEY= model_4_192_adam_bert_t1
#############

###############
model_type= bert_t1_adam
#############
inp_layer (None, 4, 192)
embedding_layer_frozen (None, 4, 192)
embedding_layer_train (None, 4, 192)
l_lstm_1f (None, 4, 512)
l_lstm_1t (None, 4, 512)
l_lstm1 (None, 8, 512)
l_conv_2 (None, 7, 128)
l_conv_3 (None, 6, 128)
l_conv_5 (None, 5, 128)
l_conv_6 (None, 3, 128)
l_conv_8 (None, 1, 128)
l_lstm_c (None, 22, 128)
l_conv_4f (None, 1, 128)
l_conv_4t (None, 1, 128)
l_conv_3f (None, 2, 128)
l_conv_3t (None, 2, 128)
l_conv_2f (None, 3, 128)
l_conv_2t (None, 3, 128)
l_merge_2 (None, 12, 128)
l_c_lstm (None, 12, 128)
Training Progress for: bert_t1_adam
Reshaping inputs
Reshaped Into Shape:  (9489, 4, 192)
Reshaped Into Shape:  (2373, 4, 192)
Reshaped Successfully!
Epoch 1/20
149/149 [==============================] - ETA: 0s - loss: 10.6256 - acc: 0.0522 - categorical_accuracy: 0.0522 - top_k_categorical_accuracy: 0.3711 - precisio

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


count_1_as_1, TP 7298
count_1_as_0, FN 1724
count_0_as_1, FP 63886
count_0_as_0, TN 137678
total_real_1s 9022
total_real_0s 201564
+--------------------+-------------+-------------+
|       Metric       | Macro Value | Micro Value |
+--------------------+-------------+-------------+
|    Hamming Loss    |    0.312    |             |
| Term Wise Accuracy |    0.688    |             |
|      Accuracy      |    0.000    |             |
|     Precision      |    0.035    |    0.103    |
|       Recall       |    0.338    |    0.809    |
|     F1-measure     |    0.059    |    0.182    |
+--------------------+-------------+-------------+
+-------------------------------+--------+
|             Metric            | Value  |
+-------------------------------+--------+
|  True Positives, count_1_as_1 |  7298  |
| False Negatives, count_1_as_0 |  1724  |
| False Positives, count_0_as_1 | 63886  |
|  True Negatives, count_0_as_0 | 137678 |
|            Real 1s            |  9022  |
|            Re

### END

## MODEL (6, 128)

In [27]:
INPUT_SHAPE =(6, 128)
EM_L_F_UNITS= 128
EM_L_T_UNITS= 128
# LEFT CHANNEL
LSTM_1F_UNITS= 128
LSTM_1T_UNITS= 128

CONV_2_FILTER= 128
CONV_2_KERNEL= 2
CONV_3_FILTER= 128
CONV_3_KERNEL= 4
CONV_5_FILTER= 128
CONV_5_KERNEL= 6
CONV_6_FILTER= 128
CONV_6_KERNEL= 8
CONV_8_FILTER= 128
CONV_8_KERNEL= 12

# RIGHT CHANNEL 
CONV_4F_FILTERS = 128
CONV_4F_KERNEL = 6
CONV_4T_FILTERS = 128
CONV_4T_KERNEL = 6

CONV_3F_FILTERS = 128
CONV_3F_KERNEL = 4
CONV_3T_FILTERS = 128
CONV_3T_KERNEL = 4

CONV_2F_FILTERS = 128
CONV_2F_KERNEL = 2
CONV_2T_FILTERS = 128
CONV_2T_KERNEL = 2

LSTM_2_C_L_UNITS = 64

OUTPUT_DENSE_UNIT =128
OUTPUT_SIZE =71
inp_shape_str = "6_128"


## RUNNING MODELS FOR 6_128

In [28]:
model_dict ={}
t_list =[x/10 for x in range(1,10)]
time_dict=[]

for ds in dataset_X.keys():
    print("\nFOR DATASET: ", ds)
    for opt in optimizer_list:
        print("FOR OPTIMIZER: ",opt)
        key= "model_"+inp_shape_str+"_"+opt+"_"+ds
        model_type = ds+"_"+opt
        print("\n###############\nKEY= "+key+"\n#############")
        print("\n###############\nmodel_type= "+model_type+"\n#############")
        model_dict[key] = BalanceNet(INPUT_SHAPE,
                                    EM_L_F_UNITS,
                                    EM_L_T_UNITS,
                                    LSTM_1F_UNITS,
                                    LSTM_1T_UNITS,
                                    CONV_2_FILTER,
                                    CONV_2_KERNEL,
                                    CONV_3_FILTER,
                                    CONV_3_KERNEL,
                                    CONV_5_FILTER,
                                    CONV_5_KERNEL,
                                    CONV_6_FILTER,
                                    CONV_6_KERNEL,
                                    CONV_8_FILTER,
                                    CONV_8_KERNEL,
                                    CONV_4F_FILTERS,
                                    CONV_4F_KERNEL,
                                    CONV_4T_FILTERS,
                                    CONV_4T_KERNEL,
                                    CONV_3F_FILTERS,
                                    CONV_3F_KERNEL,
                                    CONV_3T_FILTERS,
                                    CONV_3T_KERNEL,
                                    CONV_2F_FILTERS,
                                    CONV_2F_KERNEL,
                                    CONV_2T_FILTERS,
                                    CONV_2T_KERNEL,
                                    LSTM_2_C_L_UNITS,
                                    OUTPUT_DENSE_UNIT,
                                    OUTPUT_SIZE,
                                    optimizer_name= opt)
        if 'bert' in ds:
            em_type='bert'
        if 'xlnet' in ds:
            em_type='xlnet'
        model_dict[key].fitModel(train_x =dataset_X[ds][0], 
                                            train_y=dataset_Y[em_type][0], 
                                            val_x= dataset_X[ds][1], 
                                            val_y = dataset_Y[em_type][1], 
                                            model_type = model_type, 
                                            epochs_count = 20 , 
                                            batch_count = 64,  time_dict=time_dict, key=key)
        model_dict[key].predictModel(test_x=dataset_X[ds][2], test_y=dataset_Y[em_type][2], threshold_list=t_list,  time_dict=time_dict, key=key)
time_logger_save(time_dict,inp_shape_str+"_time")



FOR DATASET:  bert_t1
FOR OPTIMIZER:  adam

###############
KEY= model_6_128_adam_bert_t1
#############

###############
model_type= bert_t1_adam
#############
inp_layer (None, 6, 128)
embedding_layer_frozen (None, 6, 128)
embedding_layer_train (None, 6, 128)
l_lstm_1f (None, 6, 256)
l_lstm_1t (None, 6, 256)
l_lstm1 (None, 12, 256)
l_conv_2 (None, 11, 128)
l_conv_3 (None, 9, 128)
l_conv_5 (None, 7, 128)
l_conv_6 (None, 5, 128)
l_conv_8 (None, 1, 128)
l_lstm_c (None, 33, 128)
l_conv_4f (None, 1, 128)
l_conv_4t (None, 1, 128)
l_conv_3f (None, 3, 128)
l_conv_3t (None, 3, 128)
l_conv_2f (None, 5, 128)
l_conv_2t (None, 5, 128)
l_merge_2 (None, 18, 128)
l_c_lstm (None, 18, 128)
Training Progress for: bert_t1_adam
Reshaping inputs
Reshaped Into Shape:  (9489, 6, 128)
Reshaped Into Shape:  (2373, 6, 128)
Reshaped Successfully!
Epoch 1/20
149/149 [==============================] - ETA: 0s - loss: 10.6015 - acc: 0.1075 - categorical_accuracy: 0.1075 - top_k_categorical_accuracy: 0.4247 - precis

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


count_1_as_1, TP 7500
count_1_as_0, FN 1522
count_0_as_1, FP 69616
count_0_as_0, TN 131948
total_real_1s 9022
total_real_0s 201564
+--------------------+-------------+-------------+
|       Metric       | Macro Value | Micro Value |
+--------------------+-------------+-------------+
|    Hamming Loss    |    0.338    |             |
| Term Wise Accuracy |    0.662    |             |
|      Accuracy      |    0.000    |             |
|     Precision      |    0.036    |    0.097    |
|       Recall       |    0.366    |    0.831    |
|     F1-measure     |    0.061    |    0.174    |
+--------------------+-------------+-------------+
+-------------------------------+--------+
|             Metric            | Value  |
+-------------------------------+--------+
|  True Positives, count_1_as_1 |  7500  |
| False Negatives, count_1_as_0 |  1522  |
| False Positives, count_0_as_1 | 69616  |
|  True Negatives, count_0_as_0 | 131948 |
|            Real 1s            |  9022  |
|            Re

# END